In [1]:
import pandas as pd
from collections import Counter
import pprint
import time
import sys
import multiprocessing

In [44]:
df1 = pd.read_csv('/Users/jaustinj/Desktop/wargaming_backup.csv')

In [87]:
df = df1.iloc[:100000,:]

In [39]:
df.head()

,Unnamed: 0,author_id,channel_id,video_id,activity,topic
0,0,J8Am9byDy0amyXA1ORdbFA,dpt1ckt1F5IQVVCtKSF3nA,--B_MXAC5AM,liked,The Legend of Heroes
1,1,J8Am9byDy0amyXA1ORdbFA,dpt1ckt1F5IQVVCtKSF3nA,--B_MXAC5AM,liked,The Legend of Heroes: Sen no Kiseki
2,2,1ELr_Tmd4t2Y_vZM-Aed_Q,Z7ZTG4hQmzn2LDHVSBW3mw,-0kx7XUuF3I,liked,Baseball
3,3,K1DHpH_dSLdImRzJRmg01A,pVm7bg6pXKo1Pr6k5kxG9A,-0n2kxkdJgM,liked,Cat
4,4,2oleotfDSocn-h0wU9VCrA,pVm7bg6pXKo1Pr6k5kxG9A,-0n2kxkdJgM,commented,Cat


In [40]:
len(df)

5997892

In [41]:
def printProgress (iteration, total, prefix = '', suffix = '', decimals = 1, barLength = 100):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        barLength   - Optional  : character length of bar (Int)
    """
    formatStr = "{0:." + str(decimals) + "f}"
    percent = formatStr.format(100 * (iteration / float(total)))
    filledLength = int(round(barLength * iteration / float(total)))
    bar = '█' * filledLength + '-' * (barLength - filledLength)
    sys.stdout.write('\r%s |%s| %s%s %s' % (prefix, bar, percent, '%', suffix)),
    if iteration == total:
        sys.stdout.write('\n')
    sys.stdout.flush()

In [70]:
def pairs_count(df, col, group_by, out_queue):
    df = df[[col, group_by]].sort_values(by=[col, group_by]).drop_duplicates().copy()
    it = df.itertuples()
    pair_finder = Counter()
    counter = 1
    total = len(df)
    
    for row in it:
        #printProgress(counter, total, prefix='Start', suffix='Complete')
        _, col, group_by = row
        inner_it = df.iloc[counter:,:].itertuples()
        
        for other_row in inner_it:
            _, other_col, other_group_by = other_row
            if group_by != other_group_by:
                break
            elif col == other_col:
                continue
            else:
                topic_pair = str(col) + '-' + str(other_col)
                pair_finder.update([topic_pair])

        counter += 1

    out_queue.put(pair_finder)


In [88]:
cpus = multiprocessing.cpu_count()
dfs = []
partitions = list(range(0,len(df)+1,int(len(df)/cpus)))
partitions[-1] = len(df)

q = multiprocessing.Queue()
procs = []

for i in range(cpus):
    partition = df.iloc[partitions[i]:partitions[i+1],:]
    p = multiprocessing.Process(target=pairs_count, args=(partition, 'topic', 'author_id', q))
    procs.append(p)
    p.start()
    
result_dict = {}

for p in procs:
    p.join()
    
for i in range(cpus):
    result_dict.update(q.get())
    


In [89]:
len(result_dict)

650

In [74]:
cpus = multiprocessing.cpu_count()
dfs = []
partitions = list(range(0,len(df)+1,int(len(df)/cpus)))
partitions[-1] = len(df)

dfs = []

for i in range(cpus):
    partition = df.iloc[partitions[i]:partitions[i+1],:]
    dfs.append(partition)

In [75]:
dfs

[      Unnamed: 0               author_id              channel_id     video_id  \
 0              0  J8Am9byDy0amyXA1ORdbFA  dpt1ckt1F5IQVVCtKSF3nA  --B_MXAC5AM   
 1              1  J8Am9byDy0amyXA1ORdbFA  dpt1ckt1F5IQVVCtKSF3nA  --B_MXAC5AM   
 2              2  1ELr_Tmd4t2Y_vZM-Aed_Q  Z7ZTG4hQmzn2LDHVSBW3mw  -0kx7XUuF3I   
 3              3  K1DHpH_dSLdImRzJRmg01A  pVm7bg6pXKo1Pr6k5kxG9A  -0n2kxkdJgM   
 4              4  2oleotfDSocn-h0wU9VCrA  pVm7bg6pXKo1Pr6k5kxG9A  -0n2kxkdJgM   
 5              5  cfVQl13xNtO-WH-z4tI5cg  pVm7bg6pXKo1Pr6k5kxG9A  -0n2kxkdJgM   
 6              6  cfVQl13xNtO-WH-z4tI5cg  pVm7bg6pXKo1Pr6k5kxG9A  -0n2kxkdJgM   
 7              7  fqVsN6kBOREG_DdTOQ2Q9Q  pVm7bg6pXKo1Pr6k5kxG9A  -0n2kxkdJgM   
 8              8  cfVQl13xNtO-WH-z4tI5cg  pVm7bg6pXKo1Pr6k5kxG9A  -0n2kxkdJgM   
 9              9  a7HL4bW8Yy49WAad6UmVuQ  pVm7bg6pXKo1Pr6k5kxG9A  -0n2kxkdJgM   
 10            10  -cOSJJVug0hzMDsLrBZKTQ  pVm7bg6pXKo1Pr6k5kxG9A  -0n2kxkdJgM   
 11            1

In [69]:
df.iloc[250:500,:]

,Unnamed: 0,author_id,channel_id,video_id,activity,topic
250,250,eK-5OPyxW2H8REy4Ws6pkA,GtS2-7ebUgGBDjhuZ3k-5g,-XlERZlTlE8,liked,Oregon
251,251,PYLtXkgc-vmHVn9757ulGQ,GtS2-7ebUgGBDjhuZ3k-5g,-XlERZlTlE8,liked,Bundy standoff
252,252,PYLtXkgc-vmHVn9757ulGQ,GtS2-7ebUgGBDjhuZ3k-5g,-XlERZlTlE8,liked,Arrest
253,253,PYLtXkgc-vmHVn9757ulGQ,GtS2-7ebUgGBDjhuZ3k-5g,-XlERZlTlE8,liked,Militia
254,254,PYLtXkgc-vmHVn9757ulGQ,GtS2-7ebUgGBDjhuZ3k-5g,-XlERZlTlE8,liked,Oregon
255,255,x0VYUJcekI6KIagd76__Pg,hCVMMuK39IvTSSFw3tODCQ,-Y7GiQM0B9M,liked,Carson City Mint
256,256,x0VYUJcekI6KIagd76__Pg,hCVMMuK39IvTSSFw3tODCQ,-Y7GiQM0B9M,liked,Silver
257,257,x0VYUJcekI6KIagd76__Pg,hCVMMuK39IvTSSFw3tODCQ,-Y7GiQM0B9M,liked,Morgan dollar
258,258,x0VYUJcekI6KIagd76__Pg,hCVMMuK39IvTSSFw3tODCQ,-Y7GiQM0B9M,liked,Numismatic Guaranty Corporation
259,259,QKACRqNF8Q1lobSKfFoNEw,xmQg80ZHQ2mHWT3we7-jpQ,-YqssFOuBpI,liked,Recruit training
